In [53]:
import dotenv
import os
import requests
from pprint import pprint

dotenv.load_dotenv("../backend/.env")
opencage_api_key = os.getenv("OPENCAGE_API_KEY")

In [54]:
# Using the API

city = "Rio"
country = "Brazil"

response = requests.get(
    f"https://api.opencagedata.com/geocode/v1/json?q={city},{country}&key={opencage_api_key}"
)

pprint(response.json())

{'documentation': 'https://opencagedata.com/api',
 'licenses': [{'name': 'see attribution guide',
               'url': 'https://opencagedata.com/credits'}],
 'rate': {'limit': 2500, 'remaining': 2461, 'reset': 1716508800},
 'results': [{'annotations': {'DMS': {'lat': "22° 54' 39.64932'' S",
                                      'lng': "43° 12' 33.74172'' W"},
                              'MGRS': '23KPQ8364765213',
                              'Maidenhead': 'GG87jc41vi',
                              'Mercator': {'x': -4810045.366,
                                           'y': -2604632.919},
                              'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=2697338#map=17/-22.91101/-43.20937',
                                      'note_url': 'https://www.openstreetmap.org/note/new#map=17/-22.91101/-43.20937&layers=N',
                                      'url': 'https://www.openstreetmap.org/?mlat=-22.91101&mlon=-43.20937#map=17/-22.91101/-43.20937'},


In [55]:
# Using the SDK

from opencage.geocoder import OpenCageGeocode

geocoder = OpenCageGeocode(opencage_api_key)

query = 'SF, CA, USA'

results = geocoder.geocode(query)

print(results[0]['geometry']['lat'])
print(results[0]['geometry']['lng'])

37.7792588
-122.4193286


In [56]:
# The response gives us the city name
results[0]['components']['city']

'San Francisco'

In [57]:
# However, the response does not always contain the city name
query = 'Shanghai, China'
results = geocoder.geocode(query)
results[0]['components']['city'] # KeyError

KeyError: 'city'

In [58]:
# That's because it is returning the municipality of Shanghai, which is classified as a state in the response and therefore does not have a city name
results[0]['components']['_type'] # state

'state'

In [59]:
# We can mitigate this by not always using the first result of the response, but by iterating over all results and checking if the type is city
# Additionally, we add "city of" to the query to increase the chances of having the city in the response

QUERY_PREFIX = 'city of'
query = 'Shanghai, China'

results = geocoder.geocode(f"{QUERY_PREFIX} {query}")
print("# results: ", len(results), "\n")

city_index = None

for i, result in enumerate(results):
    print(result['components']['_type'])
    if result['components']['_type'] == 'city':
        city_index = i
        break
        
if city_index is None:
    raise Exception("City not found in response")

print()
print("latitude:", results[city_index]['geometry']['lat'])
print("longitude:", results[city_index]['geometry']['lng'])
print("type:", results[city_index]['components']['_type'])
print("city name:", results[city_index]['components']['_normalized_city'])


# results:  2 

attraction
city

latitude: 31.22222
longitude: 121.45806
type: city
city name: Shanghai


In [60]:
results[city_index]

{'annotations': {'DMS': {'lat': "31° 13' 19.99200'' N",
   'lng': "121° 27' 29.01600'' E"},
  'MGRS': '51RUQ5313155254',
  'Maidenhead': 'PM01rf43xh',
  'Mercator': {'x': 13520649.392, 'y': 3639496.398},
  'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/31.22222/121.45806&layers=N',
   'url': 'https://www.openstreetmap.org/?mlat=31.22222&mlon=121.45806#map=17/31.22222/121.45806'},
  'UN_M49': {'regions': {'ASIA': '142',
    'CN': '156',
    'EASTERN_ASIA': '030',
    'WORLD': '001'},
   'statistical_groupings': ['LEDC']},
  'callingcode': 86,
  'currency': {'alternate_symbols': ['CN¥', '元', 'CN元'],
   'decimal_mark': '.',
   'html_entity': '￥',
   'iso_code': 'CNY',
   'iso_numeric': '156',
   'name': 'Chinese Renminbi Yuan',
   'smallest_denomination': 1,
   'subunit': 'Fen',
   'subunit_to_unit': 100,
   'symbol': '¥',
   'symbol_first': 1,
   'thousands_separator': ','},
  'flag': '🇨🇳',
  'geohash': 'wtw3egg49gtsvqt7np5t',
  'qibla': 284.73,
  'roadinfo': {'drive_